In [1]:
import sys; sys.dont_write_bytecode=1
import warnings
from typing import Literal as Lit
import itertools as it
from numpy import nan, log, exp
import numpy as np
import pandas as pd
import deps as _
import Common
from importlib import reload; reload(_)

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
TICKERS     = _.BEST_20
TOP         = _.TOP_500
TF, SFX, SRC, UNTIL, GRAN, ANUM = _.PRESET('Y')
PRDS, SLGS  = _.aPrds(TF, [1,2,3,4,5,6]).tolist(), _.aSlug(TF, [1,2,3,4,5,6]).tolist()


LI_RNK_1 = Common.LI_RNK_1[TF]
LI_RNK_2 = Common.LI_RNK_2[TF]
TP_RNK_1 = Common.TP_RNK_1[TF]
TP_RNK_2 = Common.TP_RNK_2[TF]

# RNK_1:TP_RNK_1  = 'High Cor 1Y' 
# RNK_2:TP_RNK_2  = 'High Gap 1Y'
# IND_1, IND_2    = _.np_remove_prefix([RNK_1, RNK_2])
WGHTS           = [1,1]
NORM_WGHTS      = True
RCOMB           = 'High Art'


DOWNLOAD, COMPUTE =  True, True
FULL_CLOCK        =  True
SYNC:_.TP_SYNC    = 'SAVE'
DIRECTORY         = 'Storage/'
RESULTS           = 'Results/'
FORMAT            = 'parquet'

# Section 1

In [3]:
def GET_HIST_DATA(TICKERS, SRC, UNTIL, start=None):
    
    Raw = _.yf_api_hdatas(TICKERS, interval=SRC, period=UNTIL, start=start)

    Src = pd.DataFrame()
    Src['N']      = Raw['N']
    Src['Ticker'] = Raw['Ticker']
    Src['Date']   = Raw['Date']
    Src['Price']  = Raw['Adj Close']
    return Src

Src = GET_HIST_DATA(TICKERS, SRC, UNTIL, start=None)
Src

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,N,Ticker,Date,Price
0,0,AAPL,2024-06-01,207.149994
1,-1,AAPL,2024-05-01,191.989594
2,-2,AAPL,2024-04-01,170.099289
3,-3,AAPL,2024-03-01,171.247726
4,-4,AAPL,2024-02-01,180.275131
...,...,...,...,...
6392,-230,MKTX,2005-04-01,10.538165
6393,-231,MKTX,2005-03-01,9.601250
6394,-232,MKTX,2005-02-01,10.159962
6395,-233,MKTX,2005-01-01,12.016600


In [4]:
def MAKE_PAIRS_KEYS(Src):
    Pairs = _.pd_make_pairs(Src['Ticker'], unique=1, prefix='Ticker.')
    Date  = Src[['Date']].drop_duplicates().sort_values('Date', ascending=False) . reset_index(drop=1)
    return pd.merge(Pairs, Date, 'cross')

Keys = MAKE_PAIRS_KEYS(Src)
Keys

,Ticker.X,Ticker.Y,Ticker.XY,Date
0,AAPL,MSFT,AAPL-MSFT,2024-06-01
1,AAPL,MSFT,AAPL-MSFT,2024-05-01
2,AAPL,MSFT,AAPL-MSFT,2024-04-01
3,AAPL,MSFT,AAPL-MSFT,2024-03-01
4,AAPL,MSFT,AAPL-MSFT,2024-02-01
...,...,...,...,...
90055,EPAM,MKTX,EPAM-MKTX,1985-05-01
90056,EPAM,MKTX,EPAM-MKTX,1985-04-01
90057,EPAM,MKTX,EPAM-MKTX,1985-03-01
90058,EPAM,MKTX,EPAM-MKTX,1985-02-01


In [5]:
def MAKE_PAIRS_SOURCE(Keys, Src, TF, FULL_CLOCK):
    
    Df = Keys.copy()
    Df['Trans']   = _.in_transit(Df, 'Date', TF, FULL_CLOCK)
    Df['Price.X'] = _.pd_relate(Keys, ['Ticker.X','Date'], Src, ['Ticker','Date'], 'Price') .add_suffix('.X')
    Df['Price.Y'] = _.pd_relate(Keys, ['Ticker.Y','Date'], Src, ['Ticker','Date'], 'Price') .add_suffix('.Y')
    Df = Df.dropna().reset_index(drop=1)
    
    Df.insert(0, 'N', _.Nth(Keys, 'Date', By='Ticker.XY', sign=(-1), asc=0, adj=(+1)))

    Df['LnPrc.X'] = _.log(Df['Price.X'])
    Df['LnPrc.Y'] = _.log(Df['Price.Y'])
    return Df

Src2 = MAKE_PAIRS_SOURCE(Keys, Src, TF, FULL_CLOCK)
Src2

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,Price.X,Price.Y,LnPrc.X,LnPrc.Y
0,0,AAPL,MSFT,AAPL-MSFT,2024-06-01,0,207.149994,432.679993,5.333443,6.069998
1,-1,AAPL,MSFT,AAPL-MSFT,2024-05-01,0,191.989594,414.382568,5.257441,6.026790
2,-2,AAPL,MSFT,AAPL-MSFT,2024-04-01,0,170.099289,388.628998,5.136382,5.962625
3,-3,AAPL,MSFT,AAPL-MSFT,2024-03-01,0,171.247726,419.962494,5.143111,6.040165
4,-4,AAPL,MSFT,AAPL-MSFT,2024-02-01,0,180.275131,412.133118,5.194484,6.021346
...,...,...,...,...,...,...,...,...,...,...
47291,-365,EPAM,MKTX,EPAM-MKTX,2012-07-01,0,16.040001,27.185593,2.775086,3.302687
47292,-366,EPAM,MKTX,EPAM-MKTX,2012-06-01,0,16.990000,23.965063,2.832625,3.176597
47293,-367,EPAM,MKTX,EPAM-MKTX,2012-05-01,0,17.500000,28.950329,2.862201,3.365582
47294,-368,EPAM,MKTX,EPAM-MKTX,2012-04-01,0,20.639999,30.761406,3.027231,3.426261


# Section 2

In [6]:
def CALCULATE_PRC_STATS(Src, K, PRDS, SLGS):
    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS

    Df = Src.loc[:, None:'LnPrc.Y'].copy()
    
    def _correlate(win, minWin):
        return _.pd_rollby_correlate('pandas',  Df,'LnPrc.Y','LnPrc.X',  By='Ticker.XY', win=win, minWin=minWin,  stp=(-1), _mlt=100, _rnd=1)

    Df['Cor']       = ''
    Df[f'Cor {S1}'] = _correlate(win=P1*K, minWin=P1*K)  # nan
    Df[f'Cor {S2}'] = _correlate(win=P2*K, minWin=P1*K)  # nan
    Df[f'Cor {S3}'] = _correlate(win=P3*K, minWin=P2*K)  # nan
    Df[f'Cor {S4}'] = _correlate(win=P4*K, minWin=P3*K)  # nan
    Df[f'Cor {S5}'] = _correlate(win=P5*K, minWin=P4*K)  # nan
    Df[f'Cor {S6}'] = _correlate(win=P6*K, minWin=P5*K)  # nan
    return Df


PSts = CALCULATE_PRC_STATS(Src2, GRAN, PRDS, SLGS)
PSts

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,Price.X,Price.Y,LnPrc.X,LnPrc.Y,Cor,Cor 1Y,Cor 3Y,Cor 5Y,Cor 10Y,Cor 20Y,Cor 40Y
0,0,AAPL,MSFT,AAPL-MSFT,2024-06-01,0,207.149994,432.679993,5.333443,6.069998,,27.0,82.8,94.8,98.0,91.6,79.7
1,-1,AAPL,MSFT,AAPL-MSFT,2024-05-01,0,191.989594,414.382568,5.257441,6.026790,,-4.1,80.8,95.1,98.0,91.2,79.5
2,-2,AAPL,MSFT,AAPL-MSFT,2024-04-01,0,170.099289,388.628998,5.136382,5.962625,,-9.8,79.8,95.5,98.0,90.9,79.4
3,-3,AAPL,MSFT,AAPL-MSFT,2024-03-01,0,171.247726,419.962494,5.143111,6.040165,,16.8,81.4,96.0,98.0,90.6,79.2
4,-4,AAPL,MSFT,AAPL-MSFT,2024-02-01,0,180.275131,412.133118,5.194484,6.021346,,55.9,84.8,96.9,98.1,90.3,79.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47291,-365,EPAM,MKTX,EPAM-MKTX,2012-07-01,0,16.040001,27.185593,2.775086,3.302687,,NaN,NaN,NaN,NaN,NaN,NaN
47292,-366,EPAM,MKTX,EPAM-MKTX,2012-06-01,0,16.990000,23.965063,2.832625,3.176597,,NaN,NaN,NaN,NaN,NaN,NaN
47293,-367,EPAM,MKTX,EPAM-MKTX,2012-05-01,0,17.500000,28.950329,2.862201,3.365582,,NaN,NaN,NaN,NaN,NaN,NaN
47294,-368,EPAM,MKTX,EPAM-MKTX,2012-04-01,0,20.639999,30.761406,3.027231,3.426261,,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def CALCULATE_VARIATIONS(Src, K, PRDS, SLGS):
    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS
    
    Df = Src.copy()

    def _Return(Val, N):
        return _.Return(Df, Val, By='Ticker.XY', N=N, stp=(-1), mult=100, rnd=2)

    Df['Vars.X+']       = ''
    Df[f'Var.X +{S1}']  = _Return('Price.X', N=P1*K)
    Df[f'Var.X +{S2}']  = _Return('Price.X', N=P2*K)
    Df[f'Var.X +{S3}']  = _Return('Price.X', N=P3*K)
    Df[f'Var.X +{S4}']  = _Return('Price.X', N=P4*K)
    Df[f'Var.X +{S5}']  = _Return('Price.X', N=P5*K)
    Df[f'Var.X +{S6}']  = _Return('Price.X', N=P6*K)

    Df['Vars.Y+']       = ''
    Df[f'Var.Y +{S1}']  = _Return('Price.Y', N=P1*K)
    Df[f'Var.Y +{S2}']  = _Return('Price.Y', N=P2*K)
    Df[f'Var.Y +{S3}']  = _Return('Price.Y', N=P3*K)
    Df[f'Var.Y +{S4}']  = _Return('Price.Y', N=P4*K)
    Df[f'Var.Y +{S5}']  = _Return('Price.Y', N=P5*K)
    Df[f'Var.Y +{S6}']  = _Return('Price.Y', N=P6*K)

    def _Variat(Val, N):
        return _.Variat(Df, Val, By='Ticker.XY', N=N, stp=(-1), mult=100, rnd=2)

    Df['Vars.X-']       = ''
    Df[f'Var.X -{S1}']  = _Variat('Price.X', N=P1*K)
    Df[f'Var.X -{S2}']  = _Variat('Price.X', N=P2*K)
    Df[f'Var.X -{S3}']  = _Variat('Price.X', N=P3*K)
    Df[f'Var.X -{S4}']  = _Variat('Price.X', N=P4*K)
    Df[f'Var.X -{S5}']  = _Variat('Price.X', N=P5*K)
    Df[f'Var.X -{S6}']  = _Variat('Price.X', N=P6*K)

    Df['Vars.Y-']       = ''
    Df[f'Var.Y -{S1}']  = _Variat('Price.Y', N=P1*K)
    Df[f'Var.Y -{S2}']  = _Variat('Price.Y', N=P2*K)
    Df[f'Var.Y -{S3}']  = _Variat('Price.Y', N=P3*K)
    Df[f'Var.Y -{S4}']  = _Variat('Price.Y', N=P4*K)
    Df[f'Var.Y -{S5}']  = _Variat('Price.Y', N=P5*K)
    Df[f'Var.Y -{S6}']  = _Variat('Price.Y', N=P6*K)

    def _pct_point(Y, X):
        return _.pct_point(Df, Y, X, geo=['X','Y','Z'], pct=['X','Y','Z'], rnd=1)

    Df['Gap']       = ''
    Df[f'Gap {S1}'] = _pct_point(f'Var.Y -{S1}',  f'Var.X -{S1}')
    Df[f'Gap {S2}'] = _pct_point(f'Var.Y -{S2}',  f'Var.X -{S2}')
    Df[f'Gap {S3}'] = _pct_point(f'Var.Y -{S3}',  f'Var.X -{S3}')
    Df[f'Gap {S4}'] = _pct_point(f'Var.Y -{S4}',  f'Var.X -{S4}')
    Df[f'Gap {S5}'] = _pct_point(f'Var.Y -{S5}',  f'Var.X -{S5}')
    Df[f'Gap {S6}'] = _pct_point(f'Var.Y -{S6}',  f'Var.X -{S6}')

    return Df


Vars = CALCULATE_VARIATIONS(Src2, GRAN, PRDS, SLGS)
Vars

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,Price.X,Price.Y,LnPrc.X,LnPrc.Y,Vars.X+,Var.X +1Y,Var.X +3Y,Var.X +5Y,Var.X +10Y,Var.X +20Y,Var.X +40Y,Vars.Y+,Var.Y +1Y,Var.Y +3Y,Var.Y +5Y,Var.Y +10Y,Var.Y +20Y,Var.Y +40Y,Vars.X-,Var.X -1Y,Var.X -3Y,Var.X -5Y,Var.X -10Y,Var.X -20Y,Var.X -40Y,Vars.Y-,Var.Y -1Y,Var.Y -3Y,Var.Y -5Y,Var.Y -10Y,Var.Y -20Y,Var.Y -40Y,Gap,Gap 1Y,Gap 3Y,Gap 5Y,Gap 10Y,Gap 20Y,Gap 40Y
0,0,AAPL,MSFT,AAPL-MSFT,2024-06-01,0,207.149994,432.679993,5.333443,6.069998,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,7.36,53.83,333.79,908.03,42066.40,NaN,,28.05,63.85,238.67,1118.93,2331.10,NaN,,19.3,6.5,-21.9,20.9,-94.2,NaN
1,-1,AAPL,MSFT,AAPL-MSFT,2024-05-01,0,191.989594,414.382568,5.257441,6.026790,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,9.04,56.97,356.27,865.46,45219.92,NaN,,27.45,70.65,252.61,1097.44,2435.12,NaN,,16.9,8.7,-22.7,24.0,-94.4,NaN
2,-2,AAPL,MSFT,AAPL-MSFT,2024-04-01,0,170.099289,388.628998,5.136382,5.962625,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,0.92,31.83,252.68,817.58,43603.71,NaN,,27.75,58.46,213.17,1038.03,2286.66,NaN,,26.6,20.2,-11.2,24.0,-94.5,NaN
3,-3,AAPL,MSFT,AAPL-MSFT,2024-03-01,0,171.247726,419.962494,5.143111,6.040165,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,4.55,42.83,275.10,915.60,41848.55,NaN,,47.13,83.15,274.75,1112.08,2603.23,NaN,,40.7,28.2,-0.1,19.3,-93.6,NaN
4,-4,AAPL,MSFT,AAPL-MSFT,2024-02-01,0,180.275131,412.133118,5.194484,6.021346,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,23.30,51.69,335.04,997.00,49819.83,NaN,,67.31,82.78,288.82,1182.24,2392.84,NaN,,35.7,20.5,-10.6,16.9,-95.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47291,-365,EPAM,MKTX,EPAM-MKTX,2012-07-01,0,16.040001,27.185593,2.775086,3.302687,,80.49,362.03,435.72,2077.37,NaN,NaN,,73.37,234.66,605.02,873.79,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
47292,-366,EPAM,MKTX,EPAM-MKTX,2012-06-01,0,16.990000,23.965063,2.832625,3.176597,,59.98,319.25,394.94,1635.02,NaN,NaN,,77.84,260.10,692.71,944.39,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
47293,-367,EPAM,MKTX,EPAM-MKTX,2012-05-01,0,17.500000,28.950329,2.862201,3.365582,,32.34,310.91,379.31,1834.40,NaN,NaN,,36.13,183.54,520.77,848.69,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
47294,-368,EPAM,MKTX,EPAM-MKTX,2012-04-01,0,20.639999,30.761406,3.027231,3.426261,,4.17,213.52,273.06,1183.87,NaN,NaN,,25.04,159.01,490.17,735.56,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def CALCULATE_VAR_STATS(Src, Vars, K, PRDS, SLGS):
    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS

    Df = Src.copy()


    def _roll_stats(algo, Val, win, minWin):
        return _.pd_roll_calc(algo, Vars, Val, By='Ticker.XY', win=win, minWin=minWin, stp=(-1), _in='p100', _out='m100', fig=1)

    def add_roll_stats(Name, algo, P, S):
        Df[f'Gap.{P} {Name}.{P1}'] = _roll_stats(algo, f'Gap {S}', win=P1*K, minWin=P1*K)
        Df[f'Gap.{P} {Name}.{P2}'] = _roll_stats(algo, f'Gap {S}', win=P2*K, minWin=P1*K)
        Df[f'Gap.{P} {Name}.{P3}'] = _roll_stats(algo, f'Gap {S}', win=P3*K, minWin=P2*K)
        Df[f'Gap.{P} {Name}.{P4}'] = _roll_stats(algo, f'Gap {S}', win=P4*K, minWin=P3*K)
        Df[f'Gap.{P} {Name}.{P5}'] = _roll_stats(algo, f'Gap {S}', win=P5*K, minWin=P4*K)
        Df[f'Gap.{P} {Name}.{P6}'] = _roll_stats(algo, f'Gap {S}', win=P6*K, minWin=P5*K)
        pass
    
    Df['Gap Avg'] = ''
    add_roll_stats('Avg', 'geo_mean', P1, S1)
    add_roll_stats('Avg', 'geo_mean', P2, S2)
    add_roll_stats('Avg', 'geo_mean', P3, S3)
    add_roll_stats('Avg', 'geo_mean', P4, S4)
    add_roll_stats('Avg', 'geo_mean', P5, S5)
    add_roll_stats('Avg', 'geo_mean', P6, S6)

    Df['Gap Dev'] = ''
    add_roll_stats('Dev', 'geo_std', P1, S1)
    add_roll_stats('Dev', 'geo_std', P2, S2)
    add_roll_stats('Dev', 'geo_std', P3, S3)
    add_roll_stats('Dev', 'geo_std', P4, S4)
    add_roll_stats('Dev', 'geo_std', P5, S5)
    add_roll_stats('Dev', 'geo_std', P6, S6)


    def _ZLevel(Val, Avg, Dev):
        return _.ZLevel(..., Vars[Val], Df[Avg], Df[Dev], Pct=[1,1,1], Log=[1,1,1], fig=3) 

    def add_ZLevel(P, S):
        Df[f'Gap.{P} Lvl.{P1}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P1}',  f'Gap.{P} Dev.{P1}')
        Df[f'Gap.{P} Lvl.{P2}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P2}',  f'Gap.{P} Dev.{P2}')
        Df[f'Gap.{P} Lvl.{P3}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P3}',  f'Gap.{P} Dev.{P3}')
        Df[f'Gap.{P} Lvl.{P4}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P4}',  f'Gap.{P} Dev.{P4}')
        Df[f'Gap.{P} Lvl.{P5}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P5}',  f'Gap.{P} Dev.{P5}')
        Df[f'Gap.{P} Lvl.{P6}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P6}',  f'Gap.{P} Dev.{P6}')
        pass

    Df['Gap Lvl'] = ''
    add_ZLevel(P1, S1)
    add_ZLevel(P2, S2)
    add_ZLevel(P3, S3)
    add_ZLevel(P4, S4)
    add_ZLevel(P5, S5)
    add_ZLevel(P6, S6)

    return Df


VSts = CALCULATE_VAR_STATS(Src2, Vars, GRAN, PRDS, SLGS)
VSts

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,Price.X,Price.Y,LnPrc.X,LnPrc.Y,Gap Avg,Gap.1 Avg.1,Gap.1 Avg.3,Gap.1 Avg.5,Gap.1 Avg.10,Gap.1 Avg.20,Gap.1 Avg.40,Gap.3 Avg.1,Gap.3 Avg.3,Gap.3 Avg.5,Gap.3 Avg.10,Gap.3 Avg.20,Gap.3 Avg.40,Gap.5 Avg.1,Gap.5 Avg.3,Gap.5 Avg.5,Gap.5 Avg.10,Gap.5 Avg.20,Gap.5 Avg.40,Gap.10 Avg.1,Gap.10 Avg.3,Gap.10 Avg.5,Gap.10 Avg.10,Gap.10 Avg.20,Gap.10 Avg.40,Gap.20 Avg.1,Gap.20 Avg.3,Gap.20 Avg.5,Gap.20 Avg.10,Gap.20 Avg.20,Gap.20 Avg.40,Gap.40 Avg.1,Gap.40 Avg.3,Gap.40 Avg.5,Gap.40 Avg.10,Gap.40 Avg.20,Gap.40 Avg.40,Gap Dev,Gap.1 Dev.1,Gap.1 Dev.3,Gap.1 Dev.5,Gap.1 Dev.10,Gap.1 Dev.20,Gap.1 Dev.40,Gap.3 Dev.1,Gap.3 Dev.3,Gap.3 Dev.5,Gap.3 Dev.10,Gap.3 Dev.20,Gap.3 Dev.40,Gap.5 Dev.1,Gap.5 Dev.3,Gap.5 Dev.5,Gap.5 Dev.10,Gap.5 Dev.20,Gap.5 Dev.40,Gap.10 Dev.1,Gap.10 Dev.3,Gap.10 Dev.5,Gap.10 Dev.10,Gap.10 Dev.20,Gap.10 Dev.40,Gap.20 Dev.1,Gap.20 Dev.3,Gap.20 Dev.5,Gap.20 Dev.10,Gap.20 Dev.20,Gap.20 Dev.40,Gap.40 Dev.1,Gap.40 Dev.3,Gap.40 Dev.5,Gap.40 Dev.10,Gap.40 Dev.20,Gap.40 Dev.40,Gap Lvl,Gap.1 Lvl.1,Gap.1 Lvl.3,Gap.1 Lvl.5,Gap.1 Lvl.10,Gap.1 Lvl.20,Gap.1 Lvl.40,Gap.3 Lvl.1,Gap.3 Lvl.3,Gap.3 Lvl.5,Gap.3 Lvl.10,Gap.3 Lvl.20,Gap.3 Lvl.40,Gap.5 Lvl.1,Gap.5 Lvl.3,Gap.5 Lvl.5,Gap.5 Lvl.10,Gap.5 Lvl.20,Gap.5 Lvl.40,Gap.10 Lvl.1,Gap.10 Lvl.3,Gap.10 Lvl.5,Gap.10 Lvl.10,Gap.10 Lvl.20,Gap.10 Lvl.40,Gap.20 Lvl.1,Gap.20 Lvl.3,Gap.20 Lvl.5,Gap.20 Lvl.10,Gap.20 Lvl.20,Gap.20 Lvl.40,Gap.40 Lvl.1,Gap.40 Lvl.3,Gap.40 Lvl.5,Gap.40 Lvl.10,Gap.40 Lvl.20,Gap.40 Lvl.40
0,0,AAPL,MSFT,AAPL-MSFT,2024-06-01,0,207.149994,432.679993,5.333443,6.069998,,18.8,3.5,-2.9,0.9,-14.4,3.1,10.9,-13.4,-7.0,3.7,-39.9,8.0,-13.8,-11.4,1.2,-1.0,-55.9,10.5,9.5,13.0,-2.0,-51.0,-78.3,-8.5,-95.6,-96.7,-96.2,-95.3,-76.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,11.5,16.4,19.5,22.5,40.0,56.6,10.8,24.9,24.1,25.2,99.4,173.0,10.5,9.6,27.4,41.6,160.5,347.8,11.2,15.7,26.8,140.0,319.0,1237.1,26.1,31.8,44.6,91.1,684.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0.039,0.936,1.156,0.825,0.987,0.325,-0.395,0.930,0.628,0.119,0.829,-0.014,-0.988,-1.376,-1.070,-0.682,0.597,-0.231,0.933,0.463,0.884,1.032,1.199,0.107,1.191,2.042,1.147,0.325,-0.679,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1,AAPL,MSFT,AAPL-MSFT,2024-05-01,0,191.989594,414.382568,5.257441,6.026790,,16.4,2.7,-2.8,0.5,-14.6,3.1,8.3,-13.7,-6.7,3.5,-40.1,8.0,-13.3,-10.9,2.4,-1.5,-56.0,10.6,5.2,12.3,-3.0,-52.3,-78.2,-8.6,-95.8,-96.7,-96.3,-95.2,-76.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,13.7,16.4,19.6,22.7,40.0,56.7,15.1,24.5,24.5,25.3,99.2,173.3,10.1,9.4,27.9,42.8,160.4,348.6,15.8,15.9,27.4,145.2,322.0,1242.1,26.3,29.7,44.0,93.0,686.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0.033,0.853,1.031,0.739,0.933,0.280,0.026,1.053,0.697,0.217,0.865,0.006,-1.193,-1.581,-1.143,-0.680,0.589,-0.239,1.121,0.672,1.014,1.065,1.207,0.117,1.232,2.034,1.137,0.234,-0.699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2,AAPL,MSFT,AAPL-MSFT,2024-04-01,0,170.099289,388.628998,5.136382,5.962625,,15.1,1.8,-2.6,0.2,-14.8,3.0,5.6,-14.0,-6.4,3.4,-40.3,8.0,-12.3,-10.4,3.6,-2.0,-56.1,10.7,2.3,11.9,-3.9,-53.6,-78.0,-8.7,-96.0,-96.8,-96.3,-95.2,-76.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,14.3,16.5,20.0,22.7,39.9,56.7,17.8,24.2,25.0,25.3,99.0,173.7,9.3,9.1,28.4,43.7,160.2,349.4,15.6,15.8,27.4,150.4,325.3,1247.3,25.2,27.7,43.4,95.1,688.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0.713,1.428,1.438,1.143,1.180,0.459,0.791,1.545,1.121,0.668,1.017,0.106,0.140,-0.103,-0.617,-0.272,0.737,-0.147,1.327,0.700,1.053,1.071,1.195,0.118,1.417,2.215,1.100,0.204,-0.709,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-3,AAPL,MSFT,AAPL-MSFT,2024-03-01,0,171.247726,419.962494,5.143111,6.040165,,13.2,0.5,-2.7,-0.0,-15.1,3.0,1.5,-14.8,-6.3,3.2,-40.4,8.0,-13.0,-10.3,4.4,-2.5,-56.2,10.8,-0.4,11.2,-4.9,-54.9,-77.9,-8.8,-96.2,-96.8,-96.3,-95.1,-76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,14.3,16.6,19.8,22.6,39.9,56.8,20.4,23.2,25.1,25.3,98.7,174.0,9.6,9.1,28.7,44.6,159.9,350.2,14.5,15.8,27.8,155.5,329.6,1252.4,28.1,25.5,42.9,98.1,690.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1.627,2.191,2.042,1.676,1.505,0.693,1.258,1.958,1.400,0.962,1.116,0.170,1.508,1.237,-0.175,0.066,0.863,-0.069,1.333,0

In [67]:
def LONG_SHORT_RETURNS(Src, Vars, PRDS, SLGS):
    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS
    
    Df = pd.DataFrame()

    # Df['N']         = Src2['N'] 
    # Df['Date']      = Src2['Date'] 
    # Df[' '] = ''
    # Df['Ticker.Y']  = Src2['Ticker.Y']
    # Df['Ticker.X']  = Src2['Ticker.X']
    # Df['  '] = ''
    # Df['Var.Y 1Y']  = Vars['Var.Y -1Y']
    # Df['Gap 1Y']    = Vars['Gap 1Y']
    # Df['Var.X 1Y']  = Vars['Var.X -1Y']
    # Df['   '] = ''

    is_ngtv_1, is_pstv_1 = (Vars[f'Gap {S1}'] < 0), (0 < Vars[f'Gap {S1}'])
    is_ngtv_2, is_pstv_2 = (Vars[f'Gap {S2}'] < 0), (0 < Vars[f'Gap {S2}'])
    is_ngtv_3, is_pstv_3 = (Vars[f'Gap {S3}'] < 0), (0 < Vars[f'Gap {S3}'])
    is_ngtv_4, is_pstv_4 = (Vars[f'Gap {S4}'] < 0), (0 < Vars[f'Gap {S4}'])
    is_ngtv_5, is_pstv_5 = (Vars[f'Gap {S5}'] < 0), (0 < Vars[f'Gap {S5}'])
    is_ngtv_6, is_pstv_6 = (Vars[f'Gap {S6}'] < 0), (0 < Vars[f'Gap {S6}'])

    Df['Long Short']  = ''
    Df[f'Long {S1}']  = np.select([is_ngtv_1, is_pstv_1], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S1}'] = np.select([is_ngtv_1, is_pstv_1], [Src['Ticker.X'], Src['Ticker.Y']], '')

    Df[f'Long {S2}']  = np.select([is_ngtv_2, is_pstv_2], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S2}'] = np.select([is_ngtv_2, is_pstv_2], [Src['Ticker.X'], Src['Ticker.Y']], '')

    Df[f'Long {S3}']  = np.select([is_ngtv_3, is_pstv_3], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S3}'] = np.select([is_ngtv_3, is_pstv_3], [Src['Ticker.X'], Src['Ticker.Y']], '')

    Df[f'Long {S4}']  = np.select([is_ngtv_4, is_pstv_4], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S4}'] = np.select([is_ngtv_4, is_pstv_4], [Src['Ticker.X'], Src['Ticker.Y']], '')

    Df[f'Long {S5}']  = np.select([is_ngtv_5, is_pstv_5], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S5}'] = np.select([is_ngtv_5, is_pstv_5], [Src['Ticker.X'], Src['Ticker.Y']], '')

    Df[f'Long {S6}']  = np.select([is_ngtv_6, is_pstv_6], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S6}'] = np.select([is_ngtv_6, is_pstv_6], [Src['Ticker.X'], Src['Ticker.Y']], '')
    
    Df['LS Vars']          = ''
    Df[f'Long Var +{S1}']  = np.select([is_ngtv_1, is_pstv_1], [Vars[f'Var.Y +{S1}'], Vars[f'Var.X +{S1}']], nan)
    Df[f'Short Var +{S1}'] = np.select([is_ngtv_1, is_pstv_1], [Vars[f'Var.X +{S1}'], Vars[f'Var.Y +{S1}']], nan)

    Df[f'Long Var +{S2}']  = np.select([is_ngtv_2, is_pstv_2], [Vars[f'Var.Y +{S2}'], Vars[f'Var.X +{S2}']], nan)
    Df[f'Short Var +{S2}'] = np.select([is_ngtv_2, is_pstv_2], [Vars[f'Var.X +{S2}'], Vars[f'Var.Y +{S2}']], nan)

    Df[f'Long Var +{S3}']  = np.select([is_ngtv_3, is_pstv_3], [Vars[f'Var.Y +{S3}'], Vars[f'Var.X +{S3}']], nan)
    Df[f'Short Var +{S3}'] = np.select([is_ngtv_3, is_pstv_3], [Vars[f'Var.X +{S3}'], Vars[f'Var.Y +{S3}']], nan)

    Df[f'Long Var +{S4}']  = np.select([is_ngtv_4, is_pstv_4], [Vars[f'Var.Y +{S4}'], Vars[f'Var.X +{S4}']], nan)
    Df[f'Short Var +{S4}'] = np.select([is_ngtv_4, is_pstv_4], [Vars[f'Var.X +{S4}'], Vars[f'Var.Y +{S4}']], nan)

    Df[f'Long Var +{S5}']  = np.select([is_ngtv_5, is_pstv_5], [Vars[f'Var.Y +{S5}'], Vars[f'Var.X +{S5}']], nan)
    Df[f'Short Var +{S5}'] = np.select([is_ngtv_5, is_pstv_5], [Vars[f'Var.X +{S5}'], Vars[f'Var.Y +{S5}']], nan)

    Df[f'Long Var +{S6}']  = np.select([is_ngtv_6, is_pstv_6], [Vars[f'Var.Y +{S6}'], Vars[f'Var.X +{S6}']], nan)
    Df[f'Short Var +{S6}'] = np.select([is_ngtv_6, is_pstv_6], [Vars[f'Var.X +{S6}'], Vars[f'Var.Y +{S6}']], nan)

    Df['LS Rtn']        = ''
    Df[f'LS Rtn +{S1}'] = ((Df[f'Long Var +{S1}'] - Df[f'Short Var +{S1}']) /2) .round(2)
    Df[f'LS Rtn +{S2}'] = ((Df[f'Long Var +{S2}'] - Df[f'Short Var +{S2}']) /2) .round(2)
    Df[f'LS Rtn +{S3}'] = ((Df[f'Long Var +{S3}'] - Df[f'Short Var +{S3}']) /2) .round(2)
    Df[f'LS Rtn +{S4}'] = ((Df[f'Long Var +{S4}'] - Df[f'Short Var +{S4}']) /2) .round(2)
    Df[f'LS Rtn +{S5}'] = ((Df[f'Long Var +{S5}'] - Df[f'Short Var +{S5}']) /2) .round(2)
    Df[f'LS Rtn +{S6}'] = ((Df[f'Long Var +{S6}'] - Df[f'Short Var +{S6}']) /2) .round(2)

    return Df


LS = LONG_SHORT_RETURNS(Src2, Vars, PRDS, SLGS)
LS

,Long Short,Long 1Y,Short 1Y,Long 3Y,Short 3Y,Long 5Y,Short 5Y,Long 10Y,Short 10Y,Long 20Y,Short 20Y,Long 40Y,Short 40Y,LS Vars,Long Var +1Y,Short Var +1Y,Long Var +3Y,Short Var +3Y,Long Var +5Y,Short Var +5Y,Long Var +10Y,Short Var +10Y,Long Var +20Y,Short Var +20Y,Long Var +40Y,Short Var +40Y,LS Rtn,LS Rtn +1Y,LS Rtn +3Y,LS Rtn +5Y,LS Rtn +10Y,LS Rtn +20Y,LS Rtn +40Y
0,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,AAPL,MSFT,AAPL,MSFT,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,MSFT,AAPL,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def CALCULATE_ABSOLUTES(Vars, VSts):
    Inds = pd.concat([ Vars.loc[:, 'Gap':None], VSts.loc[:, 'Gap Avg':None] ], axis=1)
    IAbs = Inds.map(pd.to_numeric, errors='coerce').abs().add_prefix('Abs ')
    return IAbs

IAbs = CALCULATE_ABSOLUTES(Vars, VSts)
IAbs

,Abs Gap,Abs Gap 1Y,Abs Gap 3Y,Abs Gap 5Y,Abs Gap 10Y,Abs Gap 20Y,Abs Gap 40Y,Abs Gap Avg,Abs Gap.1 Avg.1,Abs Gap.1 Avg.3,Abs Gap.1 Avg.5,Abs Gap.1 Avg.10,Abs Gap.1 Avg.20,Abs Gap.1 Avg.40,Abs Gap.3 Avg.1,Abs Gap.3 Avg.3,Abs Gap.3 Avg.5,Abs Gap.3 Avg.10,Abs Gap.3 Avg.20,Abs Gap.3 Avg.40,Abs Gap.5 Avg.1,Abs Gap.5 Avg.3,Abs Gap.5 Avg.5,Abs Gap.5 Avg.10,Abs Gap.5 Avg.20,Abs Gap.5 Avg.40,Abs Gap.10 Avg.1,Abs Gap.10 Avg.3,Abs Gap.10 Avg.5,Abs Gap.10 Avg.10,Abs Gap.10 Avg.20,Abs Gap.10 Avg.40,Abs Gap.20 Avg.1,Abs Gap.20 Avg.3,Abs Gap.20 Avg.5,Abs Gap.20 Avg.10,Abs Gap.20 Avg.20,Abs Gap.20 Avg.40,Abs Gap.40 Avg.1,Abs Gap.40 Avg.3,Abs Gap.40 Avg.5,Abs Gap.40 Avg.10,Abs Gap.40 Avg.20,Abs Gap.40 Avg.40,Abs Gap Dev,Abs Gap.1 Dev.1,Abs Gap.1 Dev.3,Abs Gap.1 Dev.5,Abs Gap.1 Dev.10,Abs Gap.1 Dev.20,Abs Gap.1 Dev.40,Abs Gap.3 Dev.1,Abs Gap.3 Dev.3,Abs Gap.3 Dev.5,Abs Gap.3 Dev.10,Abs Gap.3 Dev.20,Abs Gap.3 Dev.40,Abs Gap.5 Dev.1,Abs Gap.5 Dev.3,Abs Gap.5 Dev.5,Abs Gap.5 Dev.10,Abs Gap.5 Dev.20,Abs Gap.5 Dev.40,Abs Gap.10 Dev.1,Abs Gap.10 Dev.3,Abs Gap.10 Dev.5,Abs Gap.10 Dev.10,Abs Gap.10 Dev.20,Abs Gap.10 Dev.40,Abs Gap.20 Dev.1,Abs Gap.20 Dev.3,Abs Gap.20 Dev.5,Abs Gap.20 Dev.10,Abs Gap.20 Dev.20,Abs Gap.20 Dev.40,Abs Gap.40 Dev.1,Abs Gap.40 Dev.3,Abs Gap.40 Dev.5,Abs Gap.40 Dev.10,Abs Gap.40 Dev.20,Abs Gap.40 Dev.40,Abs Gap Lvl,Abs Gap.1 Lvl.1,Abs Gap.1 Lvl.3,Abs Gap.1 Lvl.5,Abs Gap.1 Lvl.10,Abs Gap.1 Lvl.20,Abs Gap.1 Lvl.40,Abs Gap.3 Lvl.1,Abs Gap.3 Lvl.3,Abs Gap.3 Lvl.5,Abs Gap.3 Lvl.10,Abs Gap.3 Lvl.20,Abs Gap.3 Lvl.40,Abs Gap.5 Lvl.1,Abs Gap.5 Lvl.3,Abs Gap.5 Lvl.5,Abs Gap.5 Lvl.10,Abs Gap.5 Lvl.20,Abs Gap.5 Lvl.40,Abs Gap.10 Lvl.1,Abs Gap.10 Lvl.3,Abs Gap.10 Lvl.5,Abs Gap.10 Lvl.10,Abs Gap.10 Lvl.20,Abs Gap.10 Lvl.40,Abs Gap.20 Lvl.1,Abs Gap.20 Lvl.3,Abs Gap.20 Lvl.5,Abs Gap.20 Lvl.10,Abs Gap.20 Lvl.20,Abs Gap.20 Lvl.40,Abs Gap.40 Lvl.1,Abs Gap.40 Lvl.3,Abs Gap.40 Lvl.5,Abs Gap.40 Lvl.10,Abs Gap.40 Lvl.20,Abs Gap.40 Lvl.40
0,NaN,19.3,6.5,21.9,20.9,94.2,NaN,NaN,18.8,3.5,2.9,0.9,14.4,3.1,10.9,13.4,7.0,3.7,39.9,8.0,13.8,11.4,1.2,1.0,55.9,10.5,9.5,13.0,2.0,51.0,78.3,8.5,95.6,96.7,96.2,95.3,76.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.5,16.4,19.5,22.5,40.0,56.6,10.8,24.9,24.1,25.2,99.4,173.0,10.5,9.6,27.4,41.6,160.5,347.8,11.2,15.7,26.8,140.0,319.0,1237.1,26.1,31.8,44.6,91.1,684.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039,0.936,1.156,0.825,0.987,0.325,0.395,0.930,0.628,0.119,0.829,0.014,0.988,1.376,1.070,0.682,0.597,0.231,0.933,0.463,0.884,1.032,1.199,0.107,1.191,2.042,1.147,0.325,0.679,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,16.9,8.7,22.7,24.0,94.4,NaN,NaN,16.4,2.7,2.8,0.5,14.6,3.1,8.3,13.7,6.7,3.5,40.1,8.0,13.3,10.9,2.4,1.5,56.0,10.6,5.2,12.3,3.0,52.3,78.2,8.6,95.8,96.7,96.3,95.2,76.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.7,16.4,19.6,22.7,40.0,56.7,15.1,24.5,24.5,25.3,99.2,173.3,10.1,9.4,27.9,42.8,160.4,348.6,15.8,15.9,27.4,145.2,322.0,1242.1,26.3,29.7,44.0,93.0,686.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033,0.853,1.031,0.739,0.933,0.280,0.026,1.053,0.697,0.217,0.865,0.006,1.193,1.581,1.143,0.680,0.589,0.239,1.121,0.672,1.014,1.065,1.207,0.117,1.232,2.034,1.137,0.234,0.699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,26.6,20.2,11.2,24.0,94.5,NaN,NaN,15.1,1.8,2.6,0.2,14.8,3.0,5.6,14.0,6.4,3.4,40.3,8.0,12.3,10.4,3.6,2.0,56.1,10.7,2.3,11.9,3.9,53.6,78.0,8.7,96.0,96.8,96.3,95.2,76.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.3,16.5,20.0,22.7,39.9,56.7,17.8,24.2,25.0,25.3,99.0,173.7,9.3,9.1,28.4,43.7,160.2,349.4,15.6,15.8,27.4,150.4,325.3,1247.3,25.2,27.7,43.4,95.1,688.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.713,1.428,1.438,1.143,1.180,0.459,0.791,1.545,1.121,0.668,1.017,0.106,0.140,0.103,0.617,0.272,0.737,0.147,1.327,0.700,1.053,1.071,1.195,0.118,1.417,2.215,1.100,0.204,0.709,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,40.7,28.2,0.1,19.3,93.6,NaN,NaN,13.2,0.5,2.7,0.0,15.1,3.0,1.5,14.8,6.3,3.2,40.4,8.0,13.0,10.3,4.4,2.5,56.2,10.8,0.4,11.2,4.9,54.9,77.9,8.8,96.2,96.8,96.3,95.1,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.3,16.6,19.8,22.6,39.9,56.8,20.4,23.2,25.1,25.3,98.7,174.0,9.6,9.1,28.7,44.6,159.

In [11]:
def CALCULATE_GAP_VARIATION(Src, IAbs, K, PRDS, SLGS):
    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS

    Df = pd.concat([ Src.loc[:, None:'Date'], IAbs.loc[:, None:f'Abs Gap {S6}'] ], axis=1)

    def add_variat_2nd(Val, G):
        Df[f'Gap.{G} Var.{P1}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P1*K, stp=(-1), rnd=2)
        Df[f'Gap.{G} Var.{P2}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P2*K, stp=(-1), rnd=2)
        Df[f'Gap.{G} Var.{P3}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P3*K, stp=(-1), rnd=2)
        Df[f'Gap.{G} Var.{P4}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P4*K, stp=(-1), rnd=2)
        Df[f'Gap.{G} Var.{P5}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P5*K, stp=(-1), rnd=2)
        Df[f'Gap.{G} Var.{P6}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P6*K, stp=(-1), rnd=2)
        pass

    Df['Gap Var'] = ''
    add_variat_2nd(f'Abs Gap {S1}', P1)
    add_variat_2nd(f'Abs Gap {S2}', P2)
    add_variat_2nd(f'Abs Gap {S3}', P3)
    add_variat_2nd(f'Abs Gap {S4}', P4)
    add_variat_2nd(f'Abs Gap {S5}', P5)
    add_variat_2nd(f'Abs Gap {S6}', P6)
    return Df


GVar = CALCULATE_GAP_VARIATION(Src2, IAbs, GRAN, PRDS, SLGS)
GVar.head()

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Abs Gap,Abs Gap 1Y,Abs Gap 3Y,Abs Gap 5Y,Abs Gap 10Y,Abs Gap 20Y,Abs Gap 40Y,Gap Var,Gap.1 Var.1,Gap.1 Var.3,Gap.1 Var.5,Gap.1 Var.10,Gap.1 Var.20,Gap.1 Var.40,Gap.3 Var.1,Gap.3 Var.3,Gap.3 Var.5,Gap.3 Var.10,Gap.3 Var.20,Gap.3 Var.40,Gap.5 Var.1,Gap.5 Var.3,Gap.5 Var.5,Gap.5 Var.10,Gap.5 Var.20,Gap.5 Var.40,Gap.10 Var.1,Gap.10 Var.3,Gap.10 Var.5,Gap.10 Var.10,Gap.10 Var.20,Gap.10 Var.40,Gap.20 Var.1,Gap.20 Var.3,Gap.20 Var.5,Gap.20 Var.10,Gap.20 Var.20,Gap.20 Var.40,Gap.40 Var.1,Gap.40 Var.3,Gap.40 Var.5,Gap.40 Var.10,Gap.40 Var.20,Gap.40 Var.40
0,0,AAPL,MSFT,AAPL-MSFT,2024-06-01,NaN,19.3,6.5,21.9,20.9,94.2,NaN,,12.34,7.38,-6.21,-5.32,-11.24,NaN,-11.69,-0.28,-16.67,-6.41,-25.84,NaN,4.46,14.68,-21.30,-22.95,-21.10,NaN,-3.28,18.41,-10.64,-38.06,-66.15,NaN,-1.32,-1.37,-1.22,6.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1,AAPL,MSFT,AAPL-MSFT,2024-05-01,NaN,16.9,8.7,22.7,24.0,94.4,NaN,,14.83,3.54,-12.63,0.52,-11.04,NaN,-8.66,3.62,-19.30,2.74,-25.34,NaN,10.04,17.64,-23.55,-20.38,-17.37,NaN,11.51,15.46,-1.82,-36.57,-69.13,NaN,-1.17,-1.42,-0.97,6.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2,AAPL,MSFT,AAPL-MSFT,2024-04-01,NaN,26.6,20.2,11.2,24.0,94.5,NaN,,22.56,4.46,9.90,17.22,-11.65,NaN,-3.99,4.61,-2.91,16.47,-2.36,NaN,-6.71,2.39,-20.40,-27.56,-21.80,NaN,12.83,23.88,-8.22,-36.61,-76.12,NaN,-1.57,-1.02,-0.97,9.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-3,AAPL,MSFT,AAPL-MSFT,2024-03-01,NaN,40.7,28.2,0.1,19.3,93.6,NaN,,39.86,15.33,23.10,18.63,-3.56,NaN,-0.54,15.60,3.39,16.86,2.64,NaN,-15.74,-3.19,-16.23,-34.23,-38.48,NaN,18.59,16.85,-16.57,-38.69,-79.12,NaN,-1.97,-1.33,-2.02,14.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-4,AAPL,MSFT,AAPL-MSFT,2024-02-01,NaN,35.7,20.5,10.6,16.9,95.0,NaN,,27.78,14.03,10.15,16.78,5.11,NaN,-5.93,10.96,-3.29,18.49,-7.87,NaN,-6.98,2.79,-15.44,-29.01,-25.32,NaN,16.78,15.40,-17.79,-40.27,-84.86,NaN,-1.22,-1.07,-1.07,16.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def COMBINE_FRAMES(Src, LS, PSts, Vars, VSts, IAbs, GVar):
    return pd.concat([
        Src, 
        PSts.loc[:, 'Cor':None], 
        LS, 
        Vars.loc[:, 'Vars.X+':None], 
        VSts.loc[:, 'Gap Avg':None], 
        IAbs, 
        GVar.loc[:, 'Gap Var':None]
    ], axis=1)

Frames = COMBINE_FRAMES(Src2, LS, PSts, Vars, VSts, IAbs, GVar)
Frames.head()

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,Price.X,Price.Y,LnPrc.X,LnPrc.Y,Cor,Cor 1Y,Cor 3Y,Cor 5Y,Cor 10Y,Cor 20Y,Cor 40Y,Long Short,Long 1Y,Short 1Y,Long 3Y,Short 3Y,Long 5Y,Short 5Y,Long 10Y,Short 10Y,Long 20Y,Short 20Y,Long 40Y,Short 40Y,LS Vars,Long Var +1Y,Short Var +1Y,Long Var +3Y,Short Var +3Y,Long Var +5Y,Short Var +5Y,Long Var +10Y,Short Var +10Y,Long Var +20Y,Short Var +20Y,Long Var +40Y,Short Var +40Y,LS Rtn,LS Rtn +1Y,LS Rtn +3Y,LS Rtn +5Y,LS Rtn +10Y,LS Rtn +20Y,LS Rtn +40Y,Vars.X+,Var.X +1Y,Var.X +3Y,Var.X +5Y,Var.X +10Y,Var.X +20Y,Var.X +40Y,Vars.Y+,Var.Y +1Y,Var.Y +3Y,Var.Y +5Y,Var.Y +10Y,Var.Y +20Y,Var.Y +40Y,Vars.X-,Var.X -1Y,Var.X -3Y,Var.X -5Y,Var.X -10Y,Var.X -20Y,Var.X -40Y,Vars.Y-,Var.Y -1Y,Var.Y -3Y,Var.Y -5Y,Var.Y -10Y,Var.Y -20Y,Var.Y -40Y,Gap,Gap 1Y,Gap 3Y,Gap 5Y,Gap 10Y,Gap 20Y,Gap 40Y,Gap Avg,Gap.1 Avg.1,Gap.1 Avg.3,Gap.1 Avg.5,Gap.1 Avg.10,Gap.1 Avg.20,Gap.1 Avg.40,Gap.3 Avg.1,Gap.3 Avg.3,Gap.3 Avg.5,Gap.3 Avg.10,Gap.3 Avg.20,Gap.3 Avg.40,Gap.5 Avg.1,Gap.5 Avg.3,Gap.5 Avg.5,Gap.5 Avg.10,Gap.5 Avg.20,Gap.5 Avg.40,Gap.10 Avg.1,Gap.10 Avg.3,Gap.10 Avg.5,Gap.10 Avg.10,Gap.10 Avg.20,Gap.10 Avg.40,Gap.20 Avg.1,Gap.20 Avg.3,Gap.20 Avg.5,Gap.20 Avg.10,Gap.20 Avg.20,Gap.20 Avg.40,Gap.40 Avg.1,Gap.40 Avg.3,Gap.40 Avg.5,Gap.40 Avg.10,Gap.40 Avg.20,Gap.40 Avg.40,Gap Dev,Gap.1 Dev.1,Gap.1 Dev.3,Gap.1 Dev.5,Gap.1 Dev.10,Gap.1 Dev.20,Gap.1 Dev.40,Gap.3 Dev.1,Gap.3 Dev.3,Gap.3 Dev.5,Gap.3 Dev.10,Gap.3 Dev.20,Gap.3 Dev.40,Gap.5 Dev.1,Gap.5 Dev.3,Gap.5 Dev.5,Gap.5 Dev.10,Gap.5 Dev.20,Gap.5 Dev.40,Gap.10 Dev.1,Gap.10 Dev.3,Gap.10 Dev.5,Gap.10 Dev.10,Gap.10 Dev.20,Gap.10 Dev.40,Gap.20 Dev.1,Gap.20 Dev.3,Gap.20 Dev.5,Gap.20 Dev.10,Gap.20 Dev.20,Gap.20 Dev.40,Gap.40 Dev.1,Gap.40 Dev.3,Gap.40 Dev.5,Gap.40 Dev.10,Gap.40 Dev.20,Gap.40 Dev.40,Gap Lvl,Gap.1 Lvl.1,Gap.1 Lvl.3,Gap.1 Lvl.5,Gap.1 Lvl.10,Gap.1 Lvl.20,Gap.1 Lvl.40,Gap.3 Lvl.1,Gap.3 Lvl.3,Gap.3 Lvl.5,Gap.3 Lvl.10,Gap.3 Lvl.20,Gap.3 Lvl.40,Gap.5 Lvl.1,Gap.5 Lvl.3,Gap.5 Lvl.5,Gap.5 Lvl.10,Gap.5 Lvl.20,Gap.5 Lvl.40,Gap.10 Lvl.1,Gap.10 Lvl.3,Gap.10 Lvl.5,Gap.10 Lvl.10,Gap.10 Lvl.20,Gap.10 Lvl.40,Gap.20 Lvl.1,Gap.20 Lvl.3,Gap.20 Lvl.5,Gap.20 Lvl.10,Gap.20 Lvl.20,Gap.20 Lvl.40,Gap.40 Lvl.1,Gap.40 Lvl.3,Gap.40 Lvl.5,Gap.40 Lvl.10,Gap.40 Lvl.20,Gap.40 Lvl.40,Abs Gap,Abs Gap 1Y,Abs Gap 3Y,Abs Gap 5Y,Abs Gap 10Y,Abs Gap 20Y,Abs Gap 40Y,Abs Gap Avg,Abs Gap.1 Avg.1,Abs Gap.1 Avg.3,Abs Gap.1 Avg.5,Abs Gap.1 Avg.10,Abs Gap.1 Avg.20,Abs Gap.1 Avg.40,Abs Gap.3 Avg.1,Abs Gap.3 Avg.3,Abs Gap.3 Avg.5,Abs Gap.3 Avg.10,Abs Gap.3 Avg.20,Abs Gap.3 Avg.40,Abs Gap.5 Avg.1,Abs Gap.5 Avg.3,Abs Gap.5 Avg.5,Abs Gap.5 Avg.10,Abs Gap.5 Avg.20,Abs Gap.5 Avg.40,Abs Gap.10 Avg.1,Abs Gap.10 Avg.3,Abs Gap.10 Avg.5,Abs Gap.10 Avg.10,Abs Gap.10 Avg.20,Abs Gap.10 Avg.40,Abs Gap.20 Avg.1,Abs Gap.20 Avg.3,Abs Gap.20 Avg.5,Abs Gap.20 Avg.10,Abs Gap.20 Avg.20,Abs Gap.20 Avg.40,Abs Gap.40 Avg.1,Abs Gap.40 Avg.3,Abs Gap.40 Avg.5,Abs Gap.40 Avg.10,Abs Gap.40 Avg.20,Abs Gap.40 Avg.40,Abs Gap Dev,Abs Gap.1 Dev.1,Abs Gap.1 Dev.3,Abs Gap.1 Dev.5,Abs Gap.1 Dev.10,Abs Gap.1 Dev.20,Abs Gap.1 Dev.40,Abs Gap.3 Dev.1,Abs Gap.3 Dev.3,Abs Gap.3 Dev.5,Abs Gap.3 Dev.10,Abs Gap.3 Dev.20,Abs Gap.3 Dev.40,Abs Gap.5 Dev.1,Abs Gap.5 Dev.3,Abs Gap.5 Dev.5,Abs Gap.5 Dev.10,Abs Gap.5 Dev.20,Abs Gap.5 Dev.40,Abs Gap.10 Dev.1,Abs Gap.10 Dev.3,Abs Gap.10 Dev.5,Abs Gap.10 Dev.10,Abs Gap.10 Dev.20,Abs Gap.10 Dev.40,Abs Gap.20 Dev.1,Abs Gap.20 Dev.3,Abs Gap.20 Dev.5,Abs Gap.20 Dev.10,Abs Gap.20 Dev.20,Abs Gap.20 Dev.40,Abs Gap.40 Dev.1,Abs Gap.40 Dev.3,Abs Gap.40 Dev.5,Abs Gap.40 Dev.10,Abs Gap.40 Dev.20,Abs Gap.40 Dev.40,Abs Gap Lvl,Abs Gap.1 Lvl.1,Abs Gap.1 Lvl.3,Abs Gap.1 Lvl.5,Abs Gap.1 Lvl.10,Abs Gap.1 Lvl.20,Abs Gap.1 Lvl.40,Abs Gap.3 Lvl.1,Abs Gap.3 Lvl.3,Abs Gap.3 Lvl.5,Abs Gap.3 Lvl.10,Abs Gap.3 Lvl.20,Abs Gap.3 Lvl.40,Abs Gap.5 Lvl.1,Abs Gap.5 Lvl.3,Abs Gap.5 Lvl.5,Abs Gap.5 Lvl.10,Abs Gap.5 Lvl.20,Abs Gap.5 Lvl.40,Abs Gap.10 Lvl.1,Abs Gap.10 Lvl.3,Abs Gap.10 Lvl.5,Abs Gap.10 Lvl.10,Abs Gap.10 Lvl.20,Abs Gap.10 Lvl.40,Abs Ga

In [13]:
def CALCULATE_RANKS(Frames, PRDS, SLGS):

    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS

    Rnk_Rtn = _.HRnk(Frames, _.pd_columns(Frames, f'LS Rtn +{S1}', f'LS Rtn +{S6}'), By='Date', prefix='High ', _as='float')
    Rnk_Rtn.insert(0, 'High LS Rtn Rank', '')

    Rnk_Cor = _.HRnk(Frames, _.pd_columns(Frames, f'Cor {S1}', f'Cor {S6}'), By='Date', prefix='High ', _as='float')
    Rnk_Cor.insert(0, 'High Cor Rank', '')
    
    Rnk_Inds = _.RANK('BANDED', Frames, feats=['Gap',None], Keys=['Ticker.X','Ticker.Y','Ticker.XY'], By='Date', method='dense')

    return Rnk_Rtn, Rnk_Cor, Rnk_Inds

    
Rnk_Rtn, Rnk_Cor, Rnk_Inds = CALCULATE_RANKS(Frames, PRDS, SLGS)

In [14]:
def COMBINE_RANKS(Src, Rnk_Rtn, Rnk_Cor, Rnk_Inds):
    return pd.concat([
        Src.loc[:, None:'Trans'], 
        Rnk_Rtn, 
        Rnk_Cor, 
        Rnk_Inds.loc[:, 'High Rank':None], 
    ], axis=1)


Ranks = COMBINE_RANKS(Src2, Rnk_Rtn, Rnk_Cor, Rnk_Inds)
Ranks

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,High LS Rtn Rank,High LS Rtn +1Y,High LS Rtn +3Y,High LS Rtn +5Y,High LS Rtn +10Y,High LS Rtn +20Y,High LS Rtn +40Y,High Cor Rank,High Cor 1Y,High Cor 3Y,High Cor 5Y,High Cor 10Y,High Cor 20Y,High Cor 40Y,High Rank,Low Rank,High Gap,Low Gap,High Gap 1Y,Low Gap 1Y,High Gap 3Y,Low Gap 3Y,High Gap 5Y,Low Gap 5Y,High Gap 10Y,Low Gap 10Y,High Gap 20Y,Low Gap 20Y,High Gap 40Y,Low Gap 40Y,High Gap Avg,Low Gap Avg,High Gap.1 Avg.1,Low Gap.1 Avg.1,High Gap.1 Avg.3,Low Gap.1 Avg.3,High Gap.1 Avg.5,Low Gap.1 Avg.5,High Gap.1 Avg.10,Low Gap.1 Avg.10,High Gap.1 Avg.20,Low Gap.1 Avg.20,High Gap.1 Avg.40,Low Gap.1 Avg.40,High Gap.3 Avg.1,Low Gap.3 Avg.1,High Gap.3 Avg.3,Low Gap.3 Avg.3,High Gap.3 Avg.5,Low Gap.3 Avg.5,High Gap.3 Avg.10,Low Gap.3 Avg.10,High Gap.3 Avg.20,Low Gap.3 Avg.20,High Gap.3 Avg.40,Low Gap.3 Avg.40,High Gap.5 Avg.1,Low Gap.5 Avg.1,High Gap.5 Avg.3,Low Gap.5 Avg.3,High Gap.5 Avg.5,Low Gap.5 Avg.5,High Gap.5 Avg.10,Low Gap.5 Avg.10,High Gap.5 Avg.20,Low Gap.5 Avg.20,High Gap.5 Avg.40,Low Gap.5 Avg.40,High Gap.10 Avg.1,Low Gap.10 Avg.1,High Gap.10 Avg.3,Low Gap.10 Avg.3,High Gap.10 Avg.5,Low Gap.10 Avg.5,High Gap.10 Avg.10,Low Gap.10 Avg.10,High Gap.10 Avg.20,Low Gap.10 Avg.20,High Gap.10 Avg.40,Low Gap.10 Avg.40,High Gap.20 Avg.1,Low Gap.20 Avg.1,High Gap.20 Avg.3,Low Gap.20 Avg.3,High Gap.20 Avg.5,Low Gap.20 Avg.5,High Gap.20 Avg.10,Low Gap.20 Avg.10,High Gap.20 Avg.20,Low Gap.20 Avg.20,High Gap.20 Avg.40,Low Gap.20 Avg.40,High Gap.40 Avg.1,Low Gap.40 Avg.1,High Gap.40 Avg.3,Low Gap.40 Avg.3,High Gap.40 Avg.5,Low Gap.40 Avg.5,High Gap.40 Avg.10,Low Gap.40 Avg.10,High Gap.40 Avg.20,Low Gap.40 Avg.20,High Gap.40 Avg.40,Low Gap.40 Avg.40,High Gap Dev,Low Gap Dev,High Gap.1 Dev.1,Low Gap.1 Dev.1,High Gap.1 Dev.3,Low Gap.1 Dev.3,High Gap.1 Dev.5,Low Gap.1 Dev.5,High Gap.1 Dev.10,Low Gap.1 Dev.10,High Gap.1 Dev.20,Low Gap.1 Dev.20,High Gap.1 Dev.40,Low Gap.1 Dev.40,High Gap.3 Dev.1,Low Gap.3 Dev.1,High Gap.3 Dev.3,Low Gap.3 Dev.3,High Gap.3 Dev.5,Low Gap.3 Dev.5,High Gap.3 Dev.10,Low Gap.3 Dev.10,High Gap.3 Dev.20,Low Gap.3 Dev.20,High Gap.3 Dev.40,Low Gap.3 Dev.40,High Gap.5 Dev.1,Low Gap.5 Dev.1,High Gap.5 Dev.3,Low Gap.5 Dev.3,High Gap.5 Dev.5,Low Gap.5 Dev.5,High Gap.5 Dev.10,Low Gap.5 Dev.10,High Gap.5 Dev.20,Low Gap.5 Dev.20,High Gap.5 Dev.40,Low Gap.5 Dev.40,High Gap.10 Dev.1,Low Gap.10 Dev.1,High Gap.10 Dev.3,Low Gap.10 Dev.3,High Gap.10 Dev.5,Low Gap.10 Dev.5,High Gap.10 Dev.10,Low Gap.10 Dev.10,High Gap.10 Dev.20,Low Gap.10 Dev.20,High Gap.10 Dev.40,Low Gap.10 Dev.40,High Gap.20 Dev.1,Low Gap.20 Dev.1,High Gap.20 Dev.3,Low Gap.20 Dev.3,High Gap.20 Dev.5,Low Gap.20 Dev.5,High Gap.20 Dev.10,Low Gap.20 Dev.10,High Gap.20 Dev.20,Low Gap.20 Dev.20,High Gap.20 Dev.40,Low Gap.20 Dev.40,High Gap.40 Dev.1,Low Gap.40 Dev.1,High Gap.40 Dev.3,Low Gap.40 Dev.3,High Gap.40 Dev.5,Low Gap.40 Dev.5,High Gap.40 Dev.10,Low Gap.40 Dev.10,High Gap.40 Dev.20,Low Gap.40 Dev.20,High Gap.40 Dev.40,Low Gap.40 Dev.40,High Gap Lvl,Low Gap Lvl,High Gap.1 Lvl.1,Low Gap.1 Lvl.1,High Gap.1 Lvl.3,Low Gap.1 Lvl.3,High Gap.1 Lvl.5,Low Gap.1 Lvl.5,High Gap.1 Lvl.10,Low Gap.1 Lvl.10,High Gap.1 Lvl.20,Low Gap.1 Lvl.20,High Gap.1 Lvl.40,Low Gap.1 Lvl.40,High Gap.3 Lvl.1,Low Gap.3 Lvl.1,High Gap.3 Lvl.3,Low Gap.3 Lvl.3,High Gap.3 Lvl.5,Low Gap.3 Lvl.5,High Gap.3 Lvl.10,Low Gap.3 Lvl.10,High Gap.3 Lvl.20,Low Gap.3 Lvl.20,High Gap.3 Lvl.40,Low Gap.3 Lvl.40,High Gap.5 Lvl.1,Low Gap.5 Lvl.1,High Gap.5 Lvl.3,Low Gap.5 Lvl.3,High Gap.5 Lvl.5,Low Gap.5 Lvl.5,High Gap.5 Lvl.10,Low Gap.5 Lvl.10,High Gap.5 Lvl.20,Low Gap.5 Lvl.20,High Gap.5 Lvl.40,Low Gap.5 Lvl.40,High Gap.10 Lvl.1,Low Gap.10 Lvl.1,High Gap.10 Lvl.3,Low Gap.10 Lvl.3,High Gap.10 Lvl.5,Low Gap.10 Lvl.5,High Gap.10 Lvl.10,Low Gap.10 Lvl.10,High Gap.10 Lvl.20,Low Gap.10 Lvl.20,High Gap.10 Lvl.40,Low Gap.10 Lvl.40,High Gap.20 Lvl.1,Low Gap.20 Lvl.1,High Gap.20 Lvl.3,Low Gap.20 Lvl.3,High Gap.20 Lvl.5,Low Gap.20 Lvl.5,High Gap.20 Lvl.10,Low Gap.20 Lvl.10,High Gap

In [15]:
def COMBINE_DATAS(Frames, Ranks):
    return pd.concat([
        Frames,
        Ranks.loc[:, 'High LS Rtn Rank':None], 
    ], axis=1)

Data = COMBINE_DATAS(Frames, Ranks)
Data

N Ticker.X Ticker.Y  Ticker.XY       Date  Trans     Price.X  \
0        0     AAPL     MSFT  AAPL-MSFT 2024-06-01      0  207.149994   
1       -1     AAPL     MSFT  AAPL-MSFT 2024-05-01      0  191.989594   
2       -2     AAPL     MSFT  AAPL-MSFT 2024-04-01      0  170.099289   
3       -3     AAPL     MSFT  AAPL-MSFT 2024-03-01      0  171.247726   
4       -4     AAPL     MSFT  AAPL-MSFT 2024-02-01      0  180.275131   
...    ...      ...      ...        ...        ...    ...         ...   
47291 -365     EPAM     MKTX  EPAM-MKTX 2012-07-01      0   16.040001   
47292 -366     EPAM     MKTX  EPAM-MKTX 2012-06-01      0   16.990000   
47293 -367     EPAM     MKTX  EPAM-MKTX 2012-05-01      0   17.500000   
47294 -368     EPAM     MKTX  EPAM-MKTX 2012-04-01      0   20.639999   
47295 -369     EPAM     MKTX  EPAM-MKTX 2012-03-01      0   20.520000   

          Price.Y   LnPrc.X   LnPrc.Y Cor  Cor 1Y  Cor 3Y  Cor 5Y  Cor 10Y  \
0      432.679993  5.333443  6.069998        27.0    82.8    94.8     98.0   
1      414.382568  5.257441  6.026790        -4.1    80.8    95.1     98.0   
2      388.628998  5.136382  5.962625        -9.8    79.8    95.5     98.0   
3      419.962494  5.143111  6.040165        16.8    81.4    96.0     98.0   
4      412.133118  5.194484  6.021346        55.9    84.8    96.9     98.1   
...           ...       ...       ...  ..     ...     ...     ...      ...   
47291   27.185593  2.775086  3.302687         NaN     NaN     NaN      NaN   
47292   23.965063  2.832625  3.176597         NaN     NaN     NaN      NaN   
47293   28.950329  2.862201  3.365582         NaN     NaN     NaN      NaN   
47294   30.761406  3.027231  3.426261         NaN     NaN     NaN      NaN   
47295   33.433197  3.021400  3.509549         NaN     NaN     NaN      NaN   

       Cor 20Y  Cor 40Y Long Short Long 1Y Short 1Y Long 3Y Short 3Y Long 5Y  \
0         91.6     79.7        NaN    AAPL     MSFT    AAPL     MSFT    MSFT   
1         91.2     79.5        NaN    AAPL     MSFT    AAPL     MSFT    MSFT   
2         90.9     79.4        NaN    AAPL     MSFT    AAPL     MSFT    MSFT   
3         90.6     79.2        NaN    AAPL     MSFT    AAPL     MSFT    MSFT   
4         90.3     79.1        NaN    AAPL     MSFT    AAPL     MSFT    MSFT   
...        ...      ...        ...     ...      ...     ...      ...     ...   
47291      NaN      NaN        NaN                                             
47292      NaN      NaN        NaN                                             
47293      NaN      NaN        NaN                                             
47294      NaN      NaN        NaN                                             
47295      NaN      NaN        NaN                                             

      Short 5Y Long 10Y Short 10Y Long 20Y Short 20Y Long 40Y Short 40Y  \
0         AAPL     AAPL      MSFT     MSFT      AAPL                      
1         AAPL     AAPL      MSFT     MSFT      AAPL                      
2         AAPL     AAPL      MSFT     MSFT      AAPL                      
3         AAPL     AAPL      MSFT     MSFT      AAPL                      
4         AAPL     AAPL      MSFT     MSFT      AAPL                      
...        ...      ...       ...      ...       ...      ...       ...   
47291                                                                     
47292                                                                     
47293                                                                     
47294                                                                     
47295                                                                     

      LS Vars  Long Var +1Y  Short Var +1Y  Long Var +3Y  Short Var +3Y  \
0                       NaN            NaN           NaN            NaN   
1                       NaN            NaN           NaN            NaN   
2                       NaN            NaN           NaN            NaN   
3                       NaN      

In [16]:
def SORT_SCREENER(Data):
    Scrn = Data.sort_values('Date', ascending=False) . reset_index(drop=1)
    Scrn['N'] = _.Nth(Scrn, 'Date', By='', sign=(-1), asc=0, adj=(+1))
    return Scrn

Scrn = SORT_SCREENER(Data)
Scrn

N Ticker.X Ticker.Y  Ticker.XY       Date  Trans     Price.X  \
0        0     AAPL     MSFT  AAPL-MSFT 2024-06-01      0  207.149994   
1        0     IDXX       EW    IDXX-EW 2024-06-01      0  506.200012   
2        0     CPRT      RMD   CPRT-RMD 2024-06-01      0   53.270000   
3        0     CPRT      WST   CPRT-WST 2024-06-01      0   53.270000   
4        0     CPRT     MPWR  CPRT-MPWR 2024-06-01      0   53.270000   
...    ...      ...      ...        ...        ...    ...         ...   
47291 -473      UNH      WST    UNH-WST 1985-01-01      1    0.156405   
47292 -473     AAPL      SHW   AAPL-SHW 1985-01-01      1    0.099919   
47293 -473     AAPL      UNH   AAPL-UNH 1985-01-01      1    0.099919   
47294 -473      UNH      SHW    UNH-SHW 1985-01-01      1    0.156405   
47295 -473     AAPL      WST   AAPL-WST 1985-01-01      1    0.099919   

          Price.Y   LnPrc.X   LnPrc.Y Cor  Cor 1Y  Cor 3Y  Cor 5Y  Cor 10Y  \
0      432.679993  5.333443  6.069998        27.0    82.8    94.8     98.0   
1       87.000000  6.226932  4.465908        72.8    52.3    65.7     96.1   
2      210.179993  3.975373  5.347964        56.4   -39.8    44.2     95.9   
3      328.170013  3.975373  5.793532        -4.9    39.0    78.3     96.4   
4      783.979980  3.975373  6.664383        86.1    87.7    92.2     97.8   
...           ...       ...       ...  ..     ...     ...     ...      ...   
47291    1.446460 -1.855305  0.369119         NaN     NaN     NaN      NaN   
47292    0.738407 -2.303395 -0.303260         NaN     NaN     NaN      NaN   
47293    0.156405 -2.303395 -1.855305         NaN     NaN     NaN      NaN   
47294    0.738407 -1.855305 -0.303260         NaN     NaN     NaN      NaN   
47295    1.446460 -2.303395  0.369119         NaN     NaN     NaN      NaN   

       Cor 20Y  Cor 40Y Long Short Long 1Y Short 1Y Long 3Y Short 3Y Long 5Y  \
0         91.6     79.7        NaN    AAPL     MSFT    AAPL     MSFT    MSFT   
1         97.4     98.3        NaN      EW     IDXX    IDXX       EW      EW   
2         97.8     97.7        NaN     RMD     CPRT     RMD     CPRT     RMD   
3         98.3     95.9        NaN     WST     CPRT     WST     CPRT     WST   
4         98.2      NaN        NaN    CPRT     MPWR    CPRT     MPWR    CPRT   
...        ...      ...        ...     ...      ...     ...      ...     ...   
47291      NaN      NaN        NaN                                             
47292      NaN      NaN        NaN                                             
47293      NaN      NaN        NaN                                             
47294      NaN      NaN        NaN                                             
47295      NaN      NaN        NaN                                             

      Short 5Y Long 10Y Short 10Y Long 20Y Short 20Y Long 40Y Short 40Y  \
0         AAPL     AAPL      MSFT     MSFT      AAPL                      
1         IDXX       EW      IDXX       EW      IDXX                      
2         CPRT      RMD      CPRT      RMD      CPRT                      
3         CPRT      WST      CPRT     CPRT       WST                      
4         MPWR     CPRT      MPWR                                         
...        ...      ...       ...      ...       ...      ...       ...   
47291                                                                     
47292                                                                     
47293                                                                     
47294                                                                     
47295                                                                     

      LS Vars  Long Var +1Y  Short Var +1Y  Long Var +3Y  Short Var +3Y  \
0                       NaN            NaN           NaN            NaN   
1                       NaN            NaN           NaN            NaN   
2                       NaN            NaN           NaN            NaN   
3                       NaN      

# Section 3

In [17]:
def SELECT(Scrn, GRAN):
    Df = Scrn[(Scrn['N']<=(-GRAN)) & (Scrn['Trans']==1)] . reset_index(drop=1)
    Df['N'] = _.Nth(Df, 'Date', By='', sign=(-1), asc=0, adj=0)
    return Df

Slc = SELECT(Scrn, GRAN)
Slc

N Ticker.X Ticker.Y  Ticker.XY       Date  Trans     Price.X  \
0     -1     IDXX     MKTX  IDXX-MKTX 2023-01-01      1  480.500000   
1     -1     INTU     ANET  INTU-ANET 2023-01-01      1  418.393829   
2     -1      SHW     POOL   SHW-POOL 2023-01-01      1  233.212646   
3     -1     POOL     EPAM  POOL-EPAM 2023-01-01      1  378.653137   
4     -1     AAPL     META  AAPL-META 2023-01-01      1  143.110992   
...   ..      ...      ...        ...        ...    ...         ...   
3748 -39      UNH      WST    UNH-WST 1985-01-01      1    0.156405   
3749 -39     AAPL      SHW   AAPL-SHW 1985-01-01      1    0.099919   
3750 -39     AAPL      UNH   AAPL-UNH 1985-01-01      1    0.099919   
3751 -39      UNH      SHW    UNH-SHW 1985-01-01      1    0.156405   
3752 -39     AAPL      WST   AAPL-WST 1985-01-01      1    0.099919   

         Price.Y   LnPrc.X   LnPrc.Y Cor  Cor 1Y  Cor 3Y  Cor 5Y  Cor 10Y  \
0     357.672943  6.174827  5.879619        91.6    34.3    69.0     94.5   
1     126.019997  6.036423  4.836441        50.5    78.0    74.8     93.0   
2     378.653137  5.451951  5.936621        68.9    96.6    98.3     98.8   
3     332.649994  5.936621  5.807091       -71.0    78.8    94.5     97.8   
4     148.812103  4.963621  5.002684        54.2     7.7    43.2     82.1   
...          ...       ...       ...  ..     ...     ...     ...      ...   
3748    1.446460 -1.855305  0.369119         NaN     NaN     NaN      NaN   
3749    0.738407 -2.303395 -0.303260         NaN     NaN     NaN      NaN   
3750    0.156405 -2.303395 -1.855305         NaN     NaN     NaN      NaN   
3751    0.738407 -1.855305 -0.303260         NaN     NaN     NaN      NaN   
3752    1.446460 -2.303395  0.369119         NaN     NaN     NaN      NaN   

      Cor 20Y  Cor 40Y Long Short Long 1Y Short 1Y Long 3Y Short 3Y Long 5Y  \
0        96.6      NaN        NaN    IDXX     MKTX    MKTX     IDXX    MKTX   
1         NaN      NaN        NaN    INTU     ANET    INTU     ANET    ANET   
2        95.1     93.3        NaN    POOL      SHW     SHW     POOL     SHW   
3        98.0      NaN        NaN    EPAM     POOL    EPAM     POOL    EPAM   
4        80.7      NaN        NaN    META     AAPL    META     AAPL    META   
...       ...      ...        ...     ...      ...     ...      ...     ...   
3748      NaN      NaN        NaN                                             
3749      NaN      NaN        NaN                                             
3750      NaN      NaN        NaN                                             
3751      NaN      NaN        NaN                                             
3752      NaN      NaN        NaN                                             

     Short 5Y Long 10Y Short 10Y Long 20Y Short 20Y Long 40Y Short 40Y  \
0        IDXX     IDXX      MKTX                                         
1        INTU                                                            
2        POOL      SHW      POOL      SHW      POOL                      
3        POOL     POOL      EPAM                                         
4        AAPL     META      AAPL                                         
...       ...      ...       ...      ...       ...      ...       ...   
3748                                                                     
3749                                                                     
3750                                                                     
3751                                                                     
3752                                                                     

     LS Vars  Long Var +1Y  Short Var +1Y  Long Var +3Y  Short Var +3Y  \
0                     7.20         -37.37           NaN            NaN   
1                    50.46         105.27           NaN            NaN   
2                    -2.53          29.93           NaN            NaN   
3                   -16.40          -2.53           NaN            NaN   
4                   1

In [18]:
def RANK_COMBINATION(Src,  Keys, By,  Ind_1, Ind_2,  WGHTS, NORM_WGHTS=False):

    WGHTS = _.TotalNorm(WGHTS, norm=NORM_WGHTS)
    Keys  = np.ravel(Keys).tolist()
    By    = np.ravel(By).tolist()
    Df    = Src[[*Keys,*By]].copy()

    Df['Rank Comb'] = ''
    Df['Art Comb']  = (Src[Ind_1] *  WGHTS[0]  +  Src[Ind_2] *  WGHTS[1])
    Df['Pwr Comb']  = (Src[Ind_1] ** WGHTS[0]  +  Src[Ind_2] ** WGHTS[1])
    Df['Geo Comb']  = (Src[Ind_1] ** WGHTS[0]  *  Src[Ind_2] ** WGHTS[1])

    Df['High Art']  = Df.groupby(By)['Art Comb'].rank(method='dense', ascending=1)
    Df['Low Art']   = Df.groupby(By)['Art Comb'].rank(method='dense', ascending=0)

    Df['High Pwr']  = Df.groupby(By)['Pwr Comb'].rank(method='dense', ascending=1)
    Df['Low Pwr']   = Df.groupby(By)['Pwr Comb'].rank(method='dense', ascending=0)

    Df['High Geo']  = Df.groupby(By)['Geo Comb'].rank(method='dense', ascending=1)
    Df['Low Geo']   = Df.groupby(By)['Geo Comb'].rank(method='dense', ascending=0)
    return Df


RnkComb = RANK_COMBINATION(Slc,  ['N','Ticker.XY'],['Date'],  RNK_1, RNK_2,  WGHTS, NORM_WGHTS)
RnkComb.head()

NameError: name 'RNK_1' is not defined

In [ ]:
def TOP_CLASSIFCATION(RnkComb, RankBy, TOP):
    return _.pd_isin_top(RnkComb, Keys=['N','Ticker.XY','Date'], Rank=RankBy, TOP=TOP,  with_RnkVal=1)

InTop = TOP_CLASSIFCATION(RnkComb, RankBy=RCOMB, TOP=_.TOP)
InTop

In [ ]:
def COMBINE_CLASS_RCOMB(InTop, RnkComb):
    return pd.concat([  InTop, RnkComb.loc[:,'Rank Comb':None]  ], axis=1)

Class_RComb = COMBINE_CLASS_RCOMB(InTop, RnkComb)
Class_RComb

In [ ]:
def INSERT_CLASS_RCOMB_IN_DFRAME(Src, Class_RComb): 
    return _.pd_insert_df(Src, 'before','High LS Rtn Rank',  Class_RComb.loc[:, 'Rnk Val':None])

DFrame = INSERT_CLASS_RCOMB_IN_DFRAME(Slc, Class_RComb)
DFrame.head()

In [ ]:
def MAKE_AGGREGATION_KEYS(Src, TOP):
    TLINE = Src['N'].sort_values(ascending=0).unique()
    return pd.DataFrame(it.product(TLINE, TOP), columns=['N', 'Top'])


Agg = MAKE_AGGREGATION_KEYS(Slc, _.TOP)
Agg

In [ ]:
def ADD_WALLET_PERFORMANCE(Agg, Data, TF, pwr, COLS):

    def _filter(Df, TL, TOP, Rtn):
        COND_TLINE = (Df['N'] == TL)
        COND_TOP   = (Df[f'InTop {TOP}'] == 1)
        COND_ALL   = (COND_TLINE & COND_TOP)
        return Df[COND_ALL][Rtn]

    def _mean(S, rnd, alt=nan):
        try:    return S.mean().round(rnd)
        except: return alt


    Agg['Count'] = Agg.apply(lambda x: (
        _filter(Data, TL=x.N, TOP=int(x.Top), Rtn=f'LS Rtn +1{TF}').count()
    ), axis=1)

    Agg['Return'] = Agg.apply(lambda x: (
        _mean(_filter(Data, TL=x.N, TOP=int(x.Top), Rtn=f'LS Rtn +1{TF}'), rnd=2)
    ), axis=1)

    Agg['CAGR']  = _.CAGR(Agg,  'Return', By='Top', stp=(-1), pwr=pwr)
    Agg['Worth'] = _.Worth(Agg, 'Return', By='Top', stp=(-1))
    Agg['Expnt'] = _.Expnt(Agg,  'Worth', By='Top', stp=(-1))

    Agg['Inds']  = ''
    Agg[COLS]    = Agg.apply(lambda x: (
        _mean(_filter(Data, TL=x.N, TOP=int(x.Top), Rtn=COLS), rnd=2)
    ), axis=1)

    return Agg[['N','Top', 'Count','Expnt','Worth','CAGR','Return', 'Inds',*COLS]]


Agg = ADD_WALLET_PERFORMANCE(Agg, DFrame, TF, ANUM, COLS=[IND_1, IND_2, RNK_1, RNK_2])
Agg.head(60)

In [ ]:
def FINAL_RESULTS(Agg, TOP, COLS):

    Top = pd.DataFrame({ 'Top':TOP })

    Top['Count'] = Agg.groupby('Top').apply(lambda By: (
        By['Count'].mean() 
    )).reset_index(drop=1) .apply(round)

    Top[['Expnt', 'Worth', 'CAGR']] = Agg.groupby('Top').apply(lambda By: (
        By[['Expnt', 'Worth', 'CAGR']].head(1)
    )).reset_index(drop=1)

    Top['Return'] = Agg.groupby('Top').apply(lambda By: (
        np.round(By['Return'].mean(), 3)
    )).reset_index(drop=1)

    Top['Inds'] = ''
    Top[COLS]   = Agg.groupby('Top').apply(lambda By: (
        np.round(By[COLS].mean(), 3)
    )).reset_index(drop=1)

    return Top

Top = FINAL_RESULTS(Agg, _.TOP, COLS=[IND_1, IND_2, RNK_1, RNK_2])
Top